In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import urllib
import re
from IPython.display import display, HTML
import HTMLParser
import itertools
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

# Start an SQL database

In [2]:
# Create a Postgresql
dbname = 'art_kickstarter3'
username = raw_input() ##change user name to match with PostgreSQL
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print engine.url 
if not database_exists(engine.url):
    create_database(engine.url)
print 'Database exist? '+ str(database_exists(engine.url))


shaoyinghuang
postgres://shaoyinghuang@localhost/art_kickstarter3
Database exist? True


In [3]:
con = None
con = psycopg2.connect(database = dbname, user = username)

cursor = con.cursor()
command = "DROP TABLE IF EXISTS art_table;"
cursor.execute(command)
con.commit()
cursor.close()


# Scraper protytpe(s) (to be updated)

In [4]:
def scraper_prototype1(proj_info,proj_prefix):
    proj_info_split = proj_info.split('"')
    for items in proj_info_split:
        if len(items)<2:
            proj_info_split.remove(items)

    proj_info_split = map(lambda x: x.lstrip(':'), proj_info_split) 
    proj_info_split = map(lambda x: x.rstrip(','), proj_info_split) 
    proj_info_dict = dict(itertools.izip_longest(*[iter(proj_info_split)] * 2, fillvalue=""))
    proj_info_dict = {proj_prefix+'_'+k: v for k,v in proj_info_dict.items()}
    #proj_info_dict = {'project_'+k: v for k,v in proj_info_dict.items()}
    
    return proj_info_dict


# Loading the project info on a page using a straper prototype and save into database.

In [10]:
for pageno in range(10):
    r = urllib.urlopen('https://www.kickstarter.com/discover/advanced?state=live&category_id=1&sort=popularity&seed=2506137&page='+str(pageno)).read()
    soup = BeautifulSoup(r, 'html.parser')

    kk = soup.find_all('div', class_="js-react-proj-card")
    len(kk)

    for i in range(len(kk)):
        whole_string = str(kk[i]).replace('&amp','').replace('&quot;','')

        # === Overall project info ===
        try:
            project_info = '"name"'+ re.findall('"name"(.+?)"creator"',whole_string)[0]
            project_info_dict = scraper_prototype1(project_info,'project')
            df1 = pd.DataFrame.from_dict(project_info_dict,orient='index').transpose()

            # === Scrap creater string ===
            creator_string = re.findall(r'\"creator\"\:\{(.+?)\}',whole_string)
            creator_info_dict = scraper_prototype1(creator_string[0],'creator')
            df2 = pd.DataFrame.from_dict(creator_info_dict,orient='index').transpose()

            # === Scrap location string ===
            location_string = re.findall(r'\"location\"\:\{(.+?)\}',whole_string)
            location_info_dict = scraper_prototype1(location_string[0],'location')
            df3 = pd.DataFrame.from_dict(location_info_dict,orient='index').transpose()

            # === Scrap profile string ===
            profile_string = re.findall(r'\"profile\"\:\{(.+?)\}',whole_string)
            profile_info_dict = scraper_prototype1(profile_string[0],'profile')
            df4 = pd.DataFrame.from_dict(profile_info_dict,orient='index').transpose()

            df = pd.concat((df1,df2,df3,df4),axis=1)
            #display(df)

            df.to_sql('art_table', engine, if_exists='append')
        except:
            print 'Exception that scraper 1 failed. pageno = '+str(pageno)+'; i='+str(i)




Exception that scraper 1 failed. pageno = 0; i=1
Exception that scraper 1 failed. pageno = 0; i=2
Exception that scraper 1 failed. pageno = 0; i=4
Exception that scraper 1 failed. pageno = 0; i=5
Exception that scraper 1 failed. pageno = 0; i=6
Exception that scraper 1 failed. pageno = 0; i=7
Exception that scraper 1 failed. pageno = 0; i=8
Exception that scraper 1 failed. pageno = 0; i=9
Exception that scraper 1 failed. pageno = 0; i=11
Exception that scraper 1 failed. pageno = 1; i=1
Exception that scraper 1 failed. pageno = 1; i=2
Exception that scraper 1 failed. pageno = 1; i=4
Exception that scraper 1 failed. pageno = 1; i=5
Exception that scraper 1 failed. pageno = 1; i=6
Exception that scraper 1 failed. pageno = 1; i=7
Exception that scraper 1 failed. pageno = 1; i=8
Exception that scraper 1 failed. pageno = 1; i=9
Exception that scraper 1 failed. pageno = 1; i=11
Exception that scraper 1 failed. pageno = 2; i=2
Exception that scraper 1 failed. pageno = 2; i=4
Exception that scr

# Check what's on the SQL database

In [11]:
sql_query = """
SELECT * FROM art_table;
"""
birth_data_from_sql = pd.read_sql_query(sql_query,con)

birth_data_from_sql.shape

(114, 60)

In [12]:
birth_data_from_sql.head(20)

,index,project_pledged,project_name,project_currency_symbol,project_currency,project_state_changed_at,project_disable_communication,project_country,project_state,project_blurb,...,profile_project_id,profile_text_color,profile_link_text_color,profile_link_url,profile_feature_image_attributes,profile_background_color,profile_state_changed_at,profile_image_urls,profile_background_image_opacity,profile_default
0,0,4266.0,Ink the Earth,$,USD,1503676058,false,US,live,A collection of circular artwork focusing on t...,...,3097506,null,null,null,{,null,1501874347,{,0.8,https://ksr-ugc.imgix.net/assets/017/794/792/6...
1,0,16489.0,Dinosaurs of the Wild West,$,USD,1503298926,false,US,live,A collection of illustrations depicting the ev...,...,3072593,null,null,null,{,null,1499977577,{,0.8,https://ksr-ugc.imgix.net/assets/017/497/408/9...
2,0,535.0,Tombstone Screamery Pins,$,USD,1503774875,false,US,live,"I scream, you scream, we all scream for spooky...",...,3108585,null,null,null,{,null,1502757852,{,0.8,https://ksr-ugc.imgix.net/assets/018/070/322/c...
3,0,4266.0,Ink the Earth,$,USD,1503676058,false,US,live,A collection of circular artwork focusing on t...,...,3097506,null,null,null,{,null,1501874347,{,0.8,https://ksr-ugc.imgix.net/assets/017/794/792/6...
4,0,16489.0,Dinosaurs of the Wild West,$,USD,1503298926,false,US,live,A collection of illustrations depicting the ev...,...,3072593,null,null,null,{,null,1499977577,{,0.8,https://ksr-ugc.imgix.net/assets/017/497/408/9...
5,0,535.0,Tombstone Screamery Pins,$,USD,1503774875,false,US,live,"I scream, you scream, we all scream for spooky...",...,3108585,null,null,null,{,null,1502757852,{,0.8,https://ksr-ugc.imgix.net/assets/018/070/322/c...
6,0,3299.0,Girls of Ghibli Enamel Pins,$,USD,1502663838,false,US,live,A set of five enamel pins inspired by the kick...,...,3106558,null,null,null,{,null,1502596929,{,0.8,https://ksr-ugc.imgix.net/assets/017/899/682/2...
7,0,2818.0,Black Cat Enamel Pins,$,USD,1501846856,false,US,live,"A lovable fat cat, turned into an enamel pin!",...,3088815,null,null,null,{,null,1501205270,{,0.8,https://ksr-ugc.imgix.net/assets/017/709/721/a...
8,0,1342.0,Voyager 4000 - Projects of Earth,£,GBP,1503236320,false,GB,live,A series of digital paintings by Bryn G Jones.,...,3105017,null,null,null,{,null,1502463526,{,0.8,https://ksr-ugc.imgix.net/assets/017/883/332/9...
9,0,10176.0,Sailor Pets Tamagotchi Enamel Pin Collection,$,USD,1501784958,false,US,live,Hard enamel pins inspired by Sailor Moon and T...,...,3084492,null,null,null,{,null,1500911902,{,0.8,https://ksr-ugc.imgix.net/assets/017/707/749/a...
